In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
from collections import Counter
import liwc
import math
import json
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import arabic_reviews

# Hotels

In [3]:
hotels_data = pd.read_csv("Hotel_labeled_all_rows_with_ratings.csv")
hotels_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,helpfulness_gold,no,review,rating,ratings_combined
0,2905523768,False,finalized,3,12/23/2020 07:25:58,somewhat_helpful,0.6598,NaN,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,5.0,pos_rating
1,2905523769,False,finalized,3,12/23/2020 07:25:58,helpful,0.6598,NaN,52854,.. .. هنا التقيت بأحد أعجب الرجال اطلاقا،شمس ا...,5.0,pos_rating
2,2905523770,False,finalized,3,12/23/2020 07:25:58,unrelated,0.6598,NaN,48057,سأعود,3.0,neutral_ratings
3,2905523771,False,finalized,3,12/23/2020 07:25:58,helpful,1.0000,NaN,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,2.0,neg_rating
4,2905523772,False,finalized,3,12/23/2020 07:54:33,helpful,1.0000,NaN,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,4.0,pos_rating


In [3]:
hotels_conf = hotels_data[(hotels_data["confidence"] == 1.0) & (hotels_data["helpfulness"] == "helpful") | (hotels_data["helpfulness"] != "helpful")]
hotels_conf.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,no,review,helpfulness_gold,rating,ratings_combined
0,2892106090,False,finalized,3,12/10/2020 22:54:18,helpful,1.0,23768,جيد. مديرة خدمة العملاء الانسه بشرى جدا متميزة...,NaN,4,pos_rating
1,2892106091,False,finalized,3,12/10/2020 22:54:18,helpful,1.0,173933,مقبول. موقع ممتاز. انتظار الاصنصير طويل لم يكن...,NaN,3,neutral_ratings
3,2892106093,False,finalized,3,12/10/2020 22:54:18,helpful,1.0,147649,“ممتازة”. كل شي رائع و نظيف و الاسرة نظيفة ومر...,NaN,5,pos_rating
5,2892106095,False,finalized,3,12/10/2020 22:55:10,helpful,1.0,37146,مخيب للأمل. الموقع - المسبح. موظفي الاستقبال -...,NaN,2,neg_rating
7,2892106097,False,finalized,3,12/10/2020 22:55:10,helpful,1.0,139505,“لاباس به”. الموظف سفير والاستاذ كريم. التلفاز...,NaN,3,neutral_ratings


In [4]:
classes = hotels_conf["ratings_combined"].map(lambda x: "pos_rating" if x == "pos_rating" else "neg_rating")

In [5]:
hotels_data = pd.read_csv("appen_translated - hotels.csv")
hotels_conf = hotels_data[(hotels_data["confidence"] == 1.0) & (hotels_data["helpfulness"] == "helpful") | (hotels_data["helpfulness"] != "helpful")]
hotels_data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,no,review,review_en
0,2892106090,False,finalized,3,12/10/2020 22:54:18,helpful,1.0000,23768,جيد. مديرة خدمة العملاء الانسه بشرى جدا متميزة...,good. Director of Customer Service Miss Bushra...
1,2892106091,False,finalized,3,12/10/2020 22:54:18,helpful,1.0000,173933,مقبول. موقع ممتاز. انتظار الاصنصير طويل لم يكن...,Acceptable. Excellent location. Waiting for a ...
2,2892106092,False,finalized,3,12/10/2020 22:54:18,helpful,0.6765,213748,جيد. الموقع ، طاقم العمل بشوشين ، مكان الافطار...,"good. Location, staff Bchocan, place a small b..."
3,2892106093,False,finalized,3,12/10/2020 22:54:18,helpful,1.0000,147649,“ممتازة”. كل شي رائع و نظيف و الاسرة نظيفة ومر...,"""Excellent"". Everything wonderful and clean an..."
4,2892106094,False,finalized,3,12/10/2020 22:55:10,helpful,0.6604,89990,“اروع فندق”. كل شي كان فندق حقيقة اكثر من رائع...,"""Finest hotel."" Everything was more than a hot..."


In [6]:
tf_idf_conf = arabic_reviews.extract_tfidf(hotels_conf["review_en"])

[nltk_data] Downloading package stopwords to /Users/Omar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Omar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Sample feature names identified :  ['.2', '1', '1.', '10', '100', '11', '12', '13', '2', '2.', '20', '24', '24-hour', '25', '3', '3.', '30', '300', '4', '4-star', '45', '5', '5-star', '500', '6']

Size of TFIDF matrix :  (1624, 1300)


In [8]:
tf_idf_conf[0].shape

(1624, 1300)

In [9]:
arabic_reviews.ex_with_best_k(tf_idf_conf[0],classes,100)

k = ----------  1
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[  0  50]
 [  0 113]]
Acc:  0.693  Prec:  0.693  Recall:  1.0  F1: 0.819
SVM
Confusion Matrix : 
[[  2  48]
 [  0 113]]
Acc:  0.706  Prec:  0.702  Recall:  1.0  F1: 0.825
Decision Tree
Confusion Matrix : 
[[  2  48]
 [  0 113]]
Acc:  0.706  Prec:  0.702  Recall:  1.0  F1: 0.825
Random Forest
Confusion Matrix : 
[[  2  48]
 [  0 113]]
Acc:  0.706  Prec:  0.702  Recall:  1.0  F1: 0.825
NN
Confusion Matrix : 
[[  2  48]
 [  0 113]]
Acc:  0.706  Prec:  0.702  Recall:  1.0  F1: 0.825
k = ----------  2
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[  0  50]
 [  0 113]]
Acc:  0.693  Prec:  0.693  Recall:  1.0  F1: 0.819
SVM
Confusion Matrix : 
[[  3  47]
 [  3 110]]
Acc:  0.693  Prec:  0.701  Recall:  0.973  F1: 0.815
Decision Tree
Confusion Matrix : 
[[  4  46]
 [  2 111]]
Acc:  0.70

Random Forest
Confusion Matrix : 
[[  8  42]
 [  0 113]]
Acc:  0.742  Prec:  0.729  Recall:  1.0  F1: 0.843
NN
Confusion Matrix : 
[[ 29  21]
 [  9 104]]
Acc:  0.816  Prec:  0.832  Recall:  0.92  F1: 0.874
k = ----------  15
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 13  37]
 [  2 111]]
Acc:  0.761  Prec:  0.75  Recall:  0.982  F1: 0.851
SVM
Confusion Matrix : 
[[ 21  29]
 [  7 106]]
Acc:  0.779  Prec:  0.785  Recall:  0.938  F1: 0.855
Decision Tree
Confusion Matrix : 
[[30 20]
 [16 97]]
Acc:  0.779  Prec:  0.829  Recall:  0.858  F1: 0.843
Random Forest
Confusion Matrix : 
[[  9  41]
 [  0 113]]
Acc:  0.748  Prec:  0.734  Recall:  1.0  F1: 0.846
NN
Confusion Matrix : 
[[ 28  22]
 [ 12 101]]
Acc:  0.791  Prec:  0.821  Recall:  0.894  F1: 0.856
k = ----------  16
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 13  37]
 [  1 112]]
Acc:  0

NN
Confusion Matrix : 
[[ 31  19]
 [ 10 103]]
Acc:  0.822  Prec:  0.844  Recall:  0.912  F1: 0.877
k = ----------  28
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 16  34]
 [  1 112]]
Acc:  0.785  Prec:  0.767  Recall:  0.991  F1: 0.865
SVM
Confusion Matrix : 
[[ 24  26]
 [  5 108]]
Acc:  0.81  Prec:  0.806  Recall:  0.956  F1: 0.874
Decision Tree
Confusion Matrix : 
[[33 17]
 [19 94]]
Acc:  0.779  Prec:  0.847  Recall:  0.832  F1: 0.839
Random Forest
Confusion Matrix : 
[[  9  41]
 [  0 113]]
Acc:  0.748  Prec:  0.734  Recall:  1.0  F1: 0.846
NN
Confusion Matrix : 
[[ 33  17]
 [ 10 103]]
Acc:  0.834  Prec:  0.858  Recall:  0.912  F1: 0.884
k = ----------  29
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 16  34]
 [  1 112]]
Acc:  0.785  Prec:  0.767  Recall:  0.991  F1: 0.865
SVM
Confusion Matrix : 
[[ 24  26]
 [  5 108]]
Acc:  0.81  Pr

Random Forest
Confusion Matrix : 
[[  2  48]
 [  1 112]]
Acc:  0.699  Prec:  0.7  Recall:  0.991  F1: 0.821
NN
Confusion Matrix : 
[[ 36  14]
 [ 10 103]]
Acc:  0.853  Prec:  0.88  Recall:  0.912  F1: 0.896
k = ----------  42
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 16  34]
 [  1 112]]
Acc:  0.785  Prec:  0.767  Recall:  0.991  F1: 0.865
SVM
Confusion Matrix : 
[[ 25  25]
 [  7 106]]
Acc:  0.804  Prec:  0.809  Recall:  0.938  F1: 0.869
Decision Tree
Confusion Matrix : 
[[33 17]
 [14 99]]
Acc:  0.81  Prec:  0.853  Recall:  0.876  F1: 0.865
Random Forest
Confusion Matrix : 
[[  2  48]
 [  0 113]]
Acc:  0.706  Prec:  0.702  Recall:  1.0  F1: 0.825
NN
Confusion Matrix : 
[[36 14]
 [14 99]]
Acc:  0.828  Prec:  0.876  Recall:  0.876  F1: 0.876
k = ----------  43
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 16  34]
 [  1 112]]
Acc:  0.785

NN
Confusion Matrix : 
[[ 37  13]
 [  7 106]]
Acc:  0.877  Prec:  0.891  Recall:  0.938  F1: 0.914
k = ----------  55
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 18  32]
 [  1 112]]
Acc:  0.798  Prec:  0.778  Recall:  0.991  F1: 0.872
SVM
Confusion Matrix : 
[[ 26  24]
 [  5 108]]
Acc:  0.822  Prec:  0.818  Recall:  0.956  F1: 0.882
Decision Tree
Confusion Matrix : 
[[31 19]
 [14 99]]
Acc:  0.798  Prec:  0.839  Recall:  0.876  F1: 0.857
Random Forest
Confusion Matrix : 
[[  4  46]
 [  0 113]]
Acc:  0.718  Prec:  0.711  Recall:  1.0  F1: 0.831
NN
Confusion Matrix : 
[[ 39  11]
 [ 10 103]]
Acc:  0.871  Prec:  0.904  Recall:  0.912  F1: 0.907
k = ----------  56
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 18  32]
 [  1 112]]
Acc:  0.798  Prec:  0.778  Recall:  0.991  F1: 0.872
SVM
Confusion Matrix : 
[[ 26  24]
 [  5 108]]
Acc:  0.822  

Random Forest
Confusion Matrix : 
[[  1  49]
 [  0 113]]
Acc:  0.699  Prec:  0.698  Recall:  1.0  F1: 0.822
NN
Confusion Matrix : 
[[ 39  11]
 [  7 106]]
Acc:  0.89  Prec:  0.906  Recall:  0.938  F1: 0.922
k = ----------  69
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 21  29]
 [  1 112]]
Acc:  0.816  Prec:  0.794  Recall:  0.991  F1: 0.882
SVM
Confusion Matrix : 
[[ 27  23]
 [  6 107]]
Acc:  0.822  Prec:  0.823  Recall:  0.947  F1: 0.881
Decision Tree
Confusion Matrix : 
[[28 22]
 [15 98]]
Acc:  0.773  Prec:  0.817  Recall:  0.867  F1: 0.841
Random Forest
Confusion Matrix : 
[[  0  50]
 [  0 113]]
Acc:  0.693  Prec:  0.693  Recall:  1.0  F1: 0.819
NN
Confusion Matrix : 
[[ 38  12]
 [ 12 101]]
Acc:  0.853  Prec:  0.894  Recall:  0.894  F1: 0.894
k = ----------  70
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 21  29]
 [  1 112]]
Acc:  

NN
Confusion Matrix : 
[[ 36  14]
 [ 11 102]]
Acc:  0.847  Prec:  0.879  Recall:  0.903  F1: 0.891
k = ----------  82
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 19  31]
 [  1 112]]
Acc:  0.804  Prec:  0.783  Recall:  0.991  F1: 0.875
SVM
Confusion Matrix : 
[[ 27  23]
 [  6 107]]
Acc:  0.822  Prec:  0.823  Recall:  0.947  F1: 0.881
Decision Tree
Confusion Matrix : 
[[ 31  19]
 [ 13 100]]
Acc:  0.804  Prec:  0.84  Recall:  0.885  F1: 0.862
Random Forest
Confusion Matrix : 
[[  1  49]
 [  0 113]]
Acc:  0.699  Prec:  0.698  Recall:  1.0  F1: 0.822
NN
Confusion Matrix : 
[[ 38  12]
 [  9 104]]
Acc:  0.871  Prec:  0.897  Recall:  0.92  F1: 0.908
k = ----------  83
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 19  31]
 [  1 112]]
Acc:  0.804  Prec:  0.783  Recall:  0.991  F1: 0.875
SVM
Confusion Matrix : 
[[ 27  23]
 [  6 107]]
Acc:  0.822

Random Forest
Confusion Matrix : 
[[  0  50]
 [  0 113]]
Acc:  0.693  Prec:  0.693  Recall:  1.0  F1: 0.819
NN
Confusion Matrix : 
[[ 37  13]
 [ 13 100]]
Acc:  0.84  Prec:  0.885  Recall:  0.885  F1: 0.885
k = ----------  96
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 21  29]
 [  1 112]]
Acc:  0.816  Prec:  0.794  Recall:  0.991  F1: 0.882
SVM
Confusion Matrix : 
[[ 28  22]
 [  6 107]]
Acc:  0.828  Prec:  0.829  Recall:  0.947  F1: 0.884
Decision Tree
Confusion Matrix : 
[[32 18]
 [19 94]]
Acc:  0.773  Prec:  0.839  Recall:  0.832  F1: 0.836
Random Forest
Confusion Matrix : 
[[  0  50]
 [  0 113]]
Acc:  0.693  Prec:  0.693  Recall:  1.0  F1: 0.819
NN
Confusion Matrix : 
[[ 37  13]
 [ 10 103]]
Acc:  0.859  Prec:  0.888  Recall:  0.912  F1: 0.9
k = ----------  97
Classes found :  ['neg_rating' 'pos_rating']

Classes converted to integers : [1 0 1 ... 1 1 1]
NB
Confusion Matrix : 
[[ 21  29]
 [  1 112]]
Acc:  0.

In [14]:
import importlib
import arabic_reviews
importlib.reload(arabic_reviews)

<module 'arabic_reviews' from '/Users/Omar/code_local/review_helpfulness/Classification Results Sklearn/code/arabic_reviews.py'>

In [16]:
arabic_reviews.top_words(tf_idf_conf[1],tf_idf_conf[0],classes,61,"star_ratings_arabic_hotels_top_words.csv")

[('patient', 46.19943832290459),
 ('acceptable', 39.49668300546176),
 ('bad', 37.70511605511885),
 ('disappointing', 31.235605908577927),
 ('weak', 29.255418222518816),
 ('exceptional', 22.358924554957515),
 ('excellent', 21.80866530052724),
 ('hassan', 11.884095067164731),
 ('gorgeous', 9.464969595385014),
 ('good', 8.650918594004072),
 ('old', 8.625224894366184),
 ('everything', 8.430802918484607),
 ('beautiful', 8.150647097193097),
 ('wonderful', 7.621571120151936),
 ('smell', 7.082644018675309),
 ('cockroach', 5.783422864073282),
 ('taking', 5.745647949034201),
 ('never', 5.509961526001241),
 ('role', 5.4093741528828545),
 ('great', 5.029512549544706),
 ('call', 5.012005273047875),
 ('alersbhn', 4.736759457354342),
 ('unacceptable', 4.705427848037784),
 ('foul', 4.577254279054032),
 ('comfortable', 4.530712399110074),
 ('thank', 4.011512898654823),
 ('comfort', 3.8783425178955495),
 ('aansah', 3.786077901950237),
 ('leaking', 3.7852269471505706),
 ('quiet', 3.720658270839296),
 ('s